<a href="https://colab.research.google.com/github/massudmiyanji/hello_world/blob/master/Multilingual_NER_Tags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements()


fatal: destination path 'notebooks' already exists and is not an empty directory.
/content/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


In [2]:
from utils import *
setup_chapter()

No GPU was detected! This notebook can be *very* slow without a GPU 🐢
Go to Runtime > Change runtime type and select a GPU hardware accelerator.
Using transformers v4.16.2
Using datasets v1.16.1


In [3]:
import pandas as pd
toks = "Jeff Dean is a computer scientist at Google in California".split()
lbls = ["B-PER", "I-PER", "O", "O", "O", "O", "O", "B-ORG", "O", "B-LOC"]
df = pd.DataFrame(data=[toks, lbls], index=['Tokens', 'Tags'])
df

,0,1,2,3,4,5,6,7,8,9
Tokens,Jeff,Dean,is,a,computer,scientist,at,Google,in,California
Tags,B-PER,I-PER,O,O,O,O,O,B-ORG,O,B-LOC


In [4]:
from datasets import get_dataset_config_names
import pandas as pd
xtreme_subsets=get_dataset_config_names('xtreme')
print(f"XTREME has {len(xtreme_subsets)} configurations")

Downloading:   0%|          | 0.00/9.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/23.1k [00:00<?, ?B/s]

XTREME has 183 configurations


In [5]:
xtreme_subsets[:10]

['XNLI',
 'tydiqa',
 'SQuAD',
 'PAN-X.af',
 'PAN-X.ar',
 'PAN-X.bg',
 'PAN-X.bn',
 'PAN-X.de',
 'PAN-X.el',
 'PAN-X.en']

In [6]:
pan_subset=[pan for pan in xtreme_subsets if pan.startswith('PAN')]
pan_subset[:4]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg', 'PAN-X.bn']

In [7]:
from datasets import load_dataset

In [8]:
# hide_output
df=load_dataset('xtreme',name='PAN-X.en')
df

Downloading:   0%|          | 0.00/234M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xtreme downloaded and prepared to /root/.cache/huggingface/datasets/xtreme/PAN-X.en/1.0.0/2fc6b63c5326cc0d1f73060649612889b3a7ed8a6605c91cecdbd228a7158b17. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
})

In [9]:
from collections import defaultdict
from datasets import DatasetDict
langs=['de','fr','it','en']
fracs=[0.629, 0.229, 0.084, 0.059]
pan_x=defaultdict(DatasetDict)

for lang,frac in zip(langs,fracs):
  ds=load_dataset('xtreme', name=f'PAN-X.{lang}')
  for i in ds:
    pan_x[lang][i]=(ds[i].shuffle(seed=0).select(range(int(frac*ds[i].num_rows))))




0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xtreme downloaded and prepared to /root/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/2fc6b63c5326cc0d1f73060649612889b3a7ed8a6605c91cecdbd228a7158b17. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xtreme downloaded and prepared to /root/.cache/huggingface/datasets/xtreme/PAN-X.it/1.0.0/2fc6b63c5326cc0d1f73060649612889b3a7ed8a6605c91cecdbd228a7158b17. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
dataframe=pd.DataFrame({lang:[pan_x[lang]['train'].num_rows] for lang in langs},index=['number of trainig samples']   )

In [11]:
dataframe

,de,fr,it,en
number of trainig samples,12580,4580,1680,1180


In [12]:
for key, value in pan_x['en']['train'][0].items():
  print(f"{key}:{value}")

tokens:["'", "''", 'Toronto', 'Lynx', "''", "'"]
ner_tags:[0, 0, 3, 4, 0, 0]
langs:['en', 'en', 'en', 'en', 'en', 'en']


In [13]:
for key, value in pan_x['en']['train'].features.items():
  print(f"{key}:{value}")

tokens:Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
ner_tags:Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER',
'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], names_file=None, id=None),
length=-1, id=None)
langs:Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)


In [14]:
tags=pan_x['de']['train'].features['ner_tags']
tags

Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], names_file=None, id=None), length=-1, id=None)

In [15]:
label_map = {1: 'O', 2: 'B-PER', 3: 'I-PER', 4: 'B-ORG', 5: 'I-ORG', 6: 'B-LOC', 7: 'I-LOC'}
default_label = 'UNK'  # unknown label

def create_tag(batch):
    return {'ner_tag_str': [label_map.get(idx, default_label) for idx in batch['ner_tags']]}

create_tag(pan_x['de']['train'][0])

{'ner_tag_str': ['UNK',
  'UNK',
  'UNK',
  'UNK',
  'I-ORG',
  'B-LOC',
  'UNK',
  'UNK',
  'I-ORG',
  'I-ORG',
  'B-LOC',
  'UNK']}

In [16]:
de_example = pd.DataFrame({
    'token': pan_x['de']['train'][0]['tokens'],
    'tags': create_tag(pan_x['de']['train'][0])['ner_tag_str']
})



In [17]:
de_example.transpose()

,0,1,2,3,4,5,6,7,8,9,10,11
token,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
tags,UNK,UNK,UNK,UNK,I-ORG,B-LOC,UNK,UNK,I-ORG,I-ORG,B-LOC,UNK


In [18]:
from collections import Counter
from collections import defaultdict
import pandas as pd

def create_tag_sum(batch):
    return {'ner_tag_str': [[label_map.get(idx, default_label) for idx in ner_tags] for ner_tags in batch['ner_tags']]}

tags_sum = []
for split in pan_x['de']:
    result = create_tag_sum(pan_x['de'][split])
    tags_sum.append((split, result))

# Filter tags starting with "B-"
b_tags = defaultdict(Counter)
for split, dataset in tags_sum:
    for row in dataset['ner_tag_str']:
        for tag in row:
            if tag.startswith("B-"):
                b_tags[split][tag] += 1

# Create a table
table = []
for split, freq in b_tags.items():
    row = [split]
    row.append(freq.get("B-ORG", 0))
    row.append(freq.get("B-LOC", 0))
    row.append(freq.get("B-PER", 0))
    table.append(row)

df = pd.DataFrame(table, columns=["Split", "ORG", "LOC", "PER"])
df.set_index("Split", inplace=True)
print(df)

             ORG   LOC   PER
Split
validation  3820  1462  4139
test        3673  1428  4335
train       7663  2964  8265


In [19]:
from transformers import AutoTokenizer
bert_model_name='bert-base-cased'
xlmr_model_name='xlm-roberta-base'
bert_tokenizer=AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer=AutoTokenizer.from_pretrained(xlmr_model_name)

text='masoud limar likes New York.'
bert_tk=bert_tokenizer(text).tokens()
xlmr_tk=xlmr_tokenizer(text).tokens()
print(f'bert:{bert_tk}')
print(f'xlmr:{xlmr_tk}')

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

bert:['[CLS]', 'ma', '##so', '##ud', 'l', '##ima', '##r', 'likes', 'New',
'York', '.', '[SEP]']
xlmr:['<s>', '▁maso', 'ud', '▁li', 'mar', '▁like', 's', '▁New', '▁York', '.',
'</s>']


In [20]:
de_example2=pan_x['de']['train'][0]
words, labels=de_example2['tokens'], de_example2['ner_tags']
print(f'words:{words}')
print(f'labels:{labels}')

words:['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der',
'polnischen', 'Woiwodschaft', 'Pommern', '.']
labels:[0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]


In [21]:
tokenized_input = xlmr_tokenizer(de_example2["tokens"], is_split_into_words=True)
tokens = xlmr_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
pd.DataFrame([tokens],index=['Tokens'])

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>


In [22]:
word_ids=tokenized_input.word_ids()
pd.DataFrame([tokens,word_ids],index=['tokens','word_ids'])

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>
word_ids,None,0,1,1,2,3,4,4,4,5,...,9,9,9,9,10,10,10,11,11,None


In [23]:

labels

[0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]

In [24]:
previous_word_idx=None
label_ids=[]

for word_idx in word_ids:
  if word_idx is None or word_idx==previous_word_idx:
      label_ids.append(-100)
  elif word_idx !=previous_word_idx:
    label_ids.append(labels[word_idx])
  previous_word_idx=word_idx

index2tag = {
    0: "O",
    1: "B-PER",
    2: "I-PER",
    3: "B-ORG",
    4: "I-ORG",
    5: "B-LOC",
    6: "I-LOC",

}
labels = [index2tag[l] if l != -100 else "IGN" for l in label_ids]
index = ["Tokens", "Word IDs", "Label IDs", "Labels"]

pd.DataFrame([tokens, word_ids, label_ids, labels], index=index)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>
Word IDs,None,0,1,1,2,3,4,4,4,5,...,9,9,9,9,10,10,10,11,11,None
Label IDs,-100,0,0,-100,0,0,5,-100,-100,6,...,5,-100,-100,-100,6,-100,-100,0,-100,-100
Labels,IGN,O,O,IGN,O,O,B-LOC,IGN,IGN,I-LOC,...,B-LOC,IGN,IGN,IGN,I-LOC,IGN,IGN,O,IGN,IGN


In [25]:
def tokenize_and_align_labels(examples):
    # Tokenize the input with truncation and split into words
    tokenized_inputs = xlmr_tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )

    labels = []
    for idx, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        previous_word_idx = None
        label_ids = []

        # Ensure that 'label' is a list
        if not isinstance(label, list):
            raise TypeError(f"Expected a list of labels, but got {type(label).__name__}.")

        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append(-100)  # Ignore sub-token predictions
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply to a single sentence example
result = tokenize_and_align_labels({"tokens": de_example2["tokens"], "ner_tags": [de_example2["ner_tags"]]})


In [26]:
result

{'input_ids': [0, 70101, 176581, 19, 142, 122, 2290, 708, 1505, 18363, 18, 23, 122, 127474, 15439, 13787, 14, 15263, 18917, 663, 6947, 19, 6, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [[-100, 0, 0, -100, 0, 0, 5, -100, -100, 6, -100, 0, 0, 5, -100, 5, -100, -100, -100, 6, -100, -100, 0, -100, -100]]}

In [27]:
def encode_panx_dataset(corpus):
    return corpus.map(tokenize_and_align_labels, batched=True,
                      remove_columns=['langs', 'ner_tags', 'tokens'])

In [28]:
panx_de_encoded=encode_panx_dataset(pan_x['de'])

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

In [29]:
from seqeval.metrics import classification_report

y_true = [["O", "O", "O", "B-MISC", "I-MISC", "I-MISC", "O"],
          ["B-PER", "I-PER", "O"]]
y_pred = [["O", "O", "B-MISC", "I-MISC", "I-MISC", "I-MISC", "O"],
          ["B-PER", "I-PER", "O"]]
print(classification_report(y_true, y_pred))



              precision    recall  f1-score   support

        MISC       0.00      0.00      0.00         1
         PER       1.00      1.00      1.00         1

   micro avg       0.50      0.50      0.50         2
   macro avg       0.50      0.50      0.50         2
weighted avg       0.50      0.50      0.50         2



In [30]:
import numpy as np

def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    labels_list, preds_list = [], []

    for batch_idx in range(batch_size):
        example_labels, example_preds = [], []
        for seq_idx in range(seq_len):
            # Ignore label IDs = -100
            if label_ids[batch_idx, seq_idx] != -100:
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])

        labels_list.append(example_labels)
        preds_list.append(example_preds)

    return preds_list, labels_list

In [31]:
from transformers import TrainingArguments
num_epochs = 3
batch_size = 24
logging_steps = len(panx_de_encoded["train"]) // batch_size
model_name = f"{xlmr_model_name}-finetuned-panx-de"
training_args = TrainingArguments(
    output_dir=model_name, log_level="error", num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size, evaluation_strategy="epoch",
    save_steps=1e6, weight_decay=0.01, disable_tqdm=False,
    logging_steps=logging_steps, push_to_hub=True)

In [32]:
from huggingface_hub import notebook_login

notebook_login()

In [33]:
from seqeval.metrics import f1_score

def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions,
                                       eval_pred.label_ids)
    return {"f1": f1_score(y_true, y_pred)}

In [34]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(xlmr_tokenizer)

In [35]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel, RobertaPreTrainedModel

class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaConfig

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        # Load model body
        self.roberta = RobertaModel(config, add_pooling_layer=False)

        # Set up token classification head
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        # Load and initialize weights
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        # Use model body to get encoder representations
        outputs = self.roberta(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, **kwargs)

        # Apply classifier to encoder representation
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)

        # Calculate losses
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        # Return model output object
        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions
        )


In [36]:
tags

Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], names_file=None, id=None), length=-1, id=None)

In [37]:
import torch
import pandas as pd
from transformers import AutoConfig, AutoTokenizer, AutoModelForTokenClassification
from datasets import ClassLabel

# Define the tags object
tags = ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'])

# Create tag mappings from the tags object
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}

# Load model configuration
xlmr_model_name = "xlm-roberta-base"  # Replace with your actual model name

xlmr_config = AutoConfig.from_pretrained(
    xlmr_model_name,
    num_labels=tags.num_classes,
    id2label=index2tag,
    label2id=tag2index
)

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

xlmr_model = AutoModelForTokenClassification.from_pretrained(
    xlmr_model_name,
    config=xlmr_config
).to(device)

# Initialize the tokenizer
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

# Define the helper function
def tag_text(text, tags, model, tokenizer):
    # Tokenize the text and convert to input IDs
    tokens = tokenizer.tokenize(text)
    inputs = tokenizer(text, return_tensors="pt", is_split_into_words=False)
    input_ids = inputs['input_ids'].to(device)

    # Get predictions as distribution over possible classes
    with torch.no_grad():
        outputs = model(input_ids)[0]

    # Take argmax to get most likely class per token
    predictions = torch.argmax(outputs, dim=2)

    # Convert to DataFrame
    preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
    return pd.DataFrame([tokens, preds], index=["Tokens", "Tags"])

# Example text
text = "Jack Sparrow loves New York!"

# Run the function and print results
result_df = tag_text(text, tags, xlmr_model, xlmr_tokenizer)
print(result_df)


Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

            0      1      2      3      4      5      6      7      8      9
Tokens  ▁Jack  ▁Spar    row  ▁love      s   ▁New  ▁York      !   None   None
Tags    B-PER  B-PER  B-PER  B-PER  B-PER  B-PER  B-PER  B-PER  B-PER  B-PER


In [38]:
def model_init():
    return (XLMRobertaForTokenClassification
            .from_pretrained(xlmr_model_name, config=xlmr_config)
            .to(device))

In [39]:
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [40]:
!git config --global user.email "massudmiyanji@gmail.com"
!git config --global user.name "massudmiyanji"
!rm -rf xlm-roberta-base-finetuned-panx-de
!git clone https://huggingface.co/MassMin/xlm-roberta-base-finetuned-panx-de



Cloning into 'xlm-roberta-base-finetuned-panx-de'...
remote: Enumerating objects: 3, done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (3/3), 1.06 KiB | 1.06 MiB/s, done.


In [1]:
import torch
from transformers import XLMRobertaConfig, XLMRobertaForTokenClassification, Trainer, TrainingArguments

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset and determine the number of labels
# Assuming panx_de_encoded["train"] is already defined and contains the "labels" key
num_labels = len(set([label for example in panx_de_encoded["train"] for label in example['labels']]))

# Load model configuration
xlmr_model_name = "xlm-roberta-base"
xlmr_config = XLMRobertaConfig.from_pretrained(xlmr_model_name, num_labels=num_labels)

# Initialize the model with the correct number of labels
model = XLMRobertaForTokenClassification.from_pretrained(
    xlmr_model_name,
    config=xlmr_config
).to(device)

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=panx_de_encoded["train"],
    eval_dataset=panx_de_encoded["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=xlmr_tokenizer
)

# Train the model
trainer.train()


NameError: name 'panx_de_encoded' is not defined

In [ ]:
torch.save(model.state_dict(), '/content/model.pth')

In [ ]:
from google.colab import files

# Download the file
files.download('/content/model.pth')


In [ ]:
# Load the saved model state dict

model = XLMRobertaForTokenClassification.from_pretrained(
    xlmr_model_name,
    config=xlmr_config
).to(device)
model.load_state_dict(torch.load('/content/model.pth'))


In [ ]:
import os

if os.path.exists('/content/model.pth'):
    print("Model file exists.")
else:
    print("Model file not found.")


In [ ]:
df = pd.DataFrame(trainer.state.log_history)[['epoch','loss' ,'eval_loss', 'eval_f1']]
df = df.rename(columns={"epoch":"Epoch","loss": "Training Loss", "eval_loss": "Validation Loss", "eval_f1":"F1"})
df['Epoch'] = df["Epoch"].apply(lambda x: round(x))
df['Training Loss'] = df["Training Loss"].ffill()
df[['Validation Loss', 'F1']] = df[['Validation Loss', 'F1']].bfill().ffill()
df.drop_duplicates()

In [ ]:
text_de = "Jeff Dean ist ein Informatiker bei Google in Kalifornien"
tag_text(text_de, tags, trainer.model, xlmr_tokenizer)

In [ ]:
import torch

# Assuming your model is a PyTorch model
model.save_pretrained('./my_model')
xlmr_tokenizer.save_pretrained('./my_model')


In [ ]:
from huggingface_hub import HfApi, HfFolder

# Define the path to the model directory
model_directory = './my_model'

# Define the model repository name and organization (if any)
repo_name = 'MassMin/Multilingual-NER-tagging'

# Create a repository on Hugging Face Hub
api = HfApi()
api.create_repo(repo_id=repo_name, repo_type='model')

# Upload model files
api.upload_folder(
    folder_path=model_directory,
    repo_id=repo_name,
    commit_message='Initial commit of my model'
)


In [ ]:
xlmr_tokenizer

In [ ]:
model = XLMRobertaForTokenClassification.from_pretrained(
    xlmr_model_name,
    config=xlmr_config
).to(device)
model.load_state_dict(torch.load('/content/model.pth'))

